In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K


Using TensorFlow backend.


In [ ]:
import pandas as pd
#load data
data_df = pd.read_csv("/content/drive/My Drive/SyncPC/Deep learning/iqfact_2019-11-19.csv")
data_df.head()


,source,reply_to,date,reactions,text,source_url,url
0,Linh Hoàng,ROOT,2019-11-19,NaN,Mấy vụ chia sẻ du lịch chỉ thấy “các cô gái” c...,/lak.beiker?rc=p&__tn__=R,https://mbasic.facebook.com/comment/replies/?c...
1,IFact,Linh Hoàng,2019-11-19,1.0,"Chắc con trai đi tốn tiền hơn ""P",/IQFact/?rc=p&__tn__=R,https://mbasic.facebook.com/comment/replies/?c...
2,Bảo Nhii,ROOT,2019-11-19,26.0,Mấy bạn dễ thương mình chia sẻ tí. Chị <e>Nguy...,/profile.php?id=100040586285309&rc=p&__tn__=R,https://mbasic.facebook.com/comment/replies/?c...
3,Thanh Y Lai,Bảo Nhii,2019-11-19,NaN,🍄,/profile.php?id=100040417210432&rc=p&__tn__=R,https://mbasic.facebook.com/comment/replies/?c...
4,Lê Duy Trọng,Bảo Nhii,2019-11-19,NaN,🌱,/profile.php?id=100040449008593&rc=p&__tn__=R,https://mbasic.facebook.com/comment/replies/?c...


In [ ]:
import re
from nltk.tokenize import word_tokenize
def preprocessing(text):
    text = re.sub(r"\<e\>[^\<]*\<\/e\>","@",text)
    text = text.lower()
    return text+"\n"
     
preprocessing("Co ha ,de dẫn m đi")

'co ha ,de dẫn m đi\n'

In [ ]:
data_df = data_df.dropna(subset=['text'])
data_df['text'] = data_df['text'].apply(lambda x: preprocessing(x))

In [ ]:
from keras.preprocessing.text import Tokenizer


char_tokenizer = Tokenizer(num_words=None, char_level=True)
char_tokenizer.fit_on_texts(data_df['text'].values)

#Only keep top 200 char
filters = list(char_tokenizer.word_counts)[200:]

seed_text_len = 15

def create_seq(data):
    length = seed_text_len
    sequences = list()
    for text in data:
        #Filer text
        result = list(filter(lambda x: x not in filters, text))
        if (len(result) == 0):
            continue
        text = ''.join(result).strip()
        for i in range(length, len(text)):
            # select sequence of tokens
            seq = text[i-length:i+1]
            # store
            sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences   
sequences_raw = create_seq(data_df['text'].values)

Total Sequences: 485618


In [ ]:
sequences_raw[:200]

['mấy vụ chia sẻ d',
 'ấy vụ chia sẻ du',
 'y vụ chia sẻ du ',
 ' vụ chia sẻ du l',
 'vụ chia sẻ du lị',
 'ụ chia sẻ du lịc',
 ' chia sẻ du lịch',
 'chia sẻ du lịch ',
 'hia sẻ du lịch c',
 'ia sẻ du lịch ch',
 'a sẻ du lịch chỉ',
 ' sẻ du lịch chỉ ',
 'sẻ du lịch chỉ t',
 'ẻ du lịch chỉ th',
 ' du lịch chỉ thấ',
 'du lịch chỉ thấy',
 'u lịch chỉ thấy ',
 ' lịch chỉ thấy “',
 'lịch chỉ thấy “c',
 'ịch chỉ thấy “cá',
 'ch chỉ thấy “các',
 'h chỉ thấy “các ',
 ' chỉ thấy “các c',
 'chỉ thấy “các cô',
 'hỉ thấy “các cô ',
 'ỉ thấy “các cô g',
 ' thấy “các cô gá',
 'thấy “các cô gái',
 'hấy “các cô gái”',
 'ấy “các cô gái” ',
 'y “các cô gái” c',
 ' “các cô gái” ch',
 '“các cô gái” chi',
 'các cô gái” chia',
 'ác cô gái” chia ',
 'c cô gái” chia s',
 ' cô gái” chia sẻ',
 'cô gái” chia sẻ ',
 'ô gái” chia sẻ c',
 ' gái” chia sẻ ch',
 'gái” chia sẻ chứ',
 'ái” chia sẻ chứ ',
 'i” chia sẻ chứ k',
 '” chia sẻ chứ k ',
 ' chia sẻ chứ k t',
 'chia sẻ chứ k th',
 'hia sẻ chứ k thấ',
 'ia sẻ chứ k

In [ ]:
# create a character mapping index
chars = sorted(list(char_tokenizer.word_counts)[:200])
mapping = dict((c, i) for i, c in enumerate(chars))
reverse_mapping = dict( (i,c) for i,c in enumerate(chars) )
def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences_raw)



In [ ]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (437056, 15) Val shape: (48562, 15)


In [ ]:
vocab

200

In [ ]:
def model_cnn(seed_text_len, vocab):
    nb_filter = 250
    DENSE_HIDDEN_UNITS = 500
    chars = Input(shape=(seed_text_len,))
    print('Build model...')
    x = Embedding(vocab, 40, input_length=seed_text_len, trainable=True)(chars)
    def max_1d(X):
        return K.max(X, axis=1)

    # we add a Convolution1D, which will learn nb_filter
    # word group filters of size 3:

    con3_layer = Convolution1D(nb_filter=nb_filter,
                        filter_length=2,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1)(x)

    pool_con3_layer = Lambda(max_1d, output_shape=(nb_filter,))(con3_layer)


    # we add a Convolution1D, which will learn nb_filter
    # word group filters of size 4:

    con4_layer = Convolution1D(nb_filter=nb_filter,
                        filter_length=3,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1)(x)

    pool_con4_layer =  Lambda(max_1d, output_shape=(nb_filter,))(con4_layer)


    # we add a Convolution1D, which will learn nb_filter
    # word group filters of size 5:

    con5_layer = Convolution1D(nb_filter=nb_filter,
                        filter_length=4,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1)(x)

    pool_con5_layer = Lambda(max_1d, output_shape=(nb_filter,))(con5_layer)



    lstm_layer = GRU(300, recurrent_dropout=0.1, dropout=0.1,activation="sigmoid")(x)

    sent_emb_layer_model = concatenate([pool_con3_layer, pool_con5_layer,pool_con4_layer,lstm_layer])
    cnn_layer = Dense(DENSE_HIDDEN_UNITS, activation='sigmoid')(sent_emb_layer_model)
    x = Dropout(0.5)(cnn_layer)
    result = Dense(vocab, activation='softmax')(x)

    model = Model(inputs=chars, outputs=result)
    model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=["accuracy"])
    return model



In [ ]:
from keras import backend as K
from keras.callbacks import ModelCheckpoint

check = ModelCheckpoint("cnn_lstm.hdf5",monitor='val_acc',save_best_only=True,save_weights_only=True )


# define model CNN
cnn_model = model_cnn(seed_text_len, vocab)
print(cnn_model.summary())

# compile the model
cnn_model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
cnn_model.fit(X_tr, y_tr, epochs=50, verbose=1, validation_data=(X_val, y_val),batch_size=4048, callbacks=[check])

Build model...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=250, kernel_size=2, strides=1, padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=250, kernel_size=3, strides=1, padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=250, kernel_size=4, strides=1, padding="valid")`


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 15, 40)       8000        input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 14, 250)      20250       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 12, 250)      40250       embedding_2[0][0]                
____________________________________________________________________________________________

In [ ]:
import random
import heapq
# generate a sequence of characters with a language model
def generate_seq(model, mapping, reverse_mapping, seq_length, seed_text, n_chars = 40):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict character
        yhat = model.predict(encoded, verbose=0)[0]
        #yhat = heapq.nlargest(5, range(len(yhat)), yhat.__getitem__)#top 5 indices        
        yhat = np.argmax(yhat,axis=1)
        #yhat = yhat[0]
        
        # reverse map integer to character
        out_char = reverse_mapping[yhat]
        
        if out_char == "\n":
            break
        # append to input
        in_text += out_char
    return in_text

In [ ]:
seed_text = "mai mốt "
print(len(seed_text))
generate_seq(cnn_model,mapping,reverse_mapping,seed_text_len,seed_text)

8


'mai mốt nghiệp nha :))))))))))))))))))))))))))))'

In [ ]:

import heapq

a = np.array([[1,4,5],[1,7,9],[5,2,6]])
y = [  heapq.nlargest(2, range(len(b)), b.__getitem__) for b in a]
y

[[2, 1], [2, 1], [2, 0]]